In [62]:
# !pip install tensorflow-gpu
# !pip install deeppavlov
# !python -m deeppavlov install squad_bert
# !pip install fasttext
# !pip install transformers
# !python -m deeppavlov install bert_sentence_embedder
# %pip install torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import torch

## Загружаем предобученую модель для анализа тональности текста

In [63]:
# https://huggingface.co/Tatyana/rubert-base-cased-sentiment-new
tokenizer = AutoTokenizer.from_pretrained("Tatyana/rubert-base-cased-sentiment-new")
model = AutoModelForSequenceClassification.from_pretrained("Tatyana/rubert-base-cased-sentiment-new")

In [64]:
def classify_text(text):
    inputs = tokenizer(text, return_tensors="pt")

    with torch.no_grad():
        logits = model(**inputs).logits

    return model.config.id2label[logits.argmax().item()]

## Делаем некоторые преобразования в данных

In [70]:
data = pd.read_csv('data/data.csv').dropna()
data = data.rename(columns={"Rating": "rating", "Review": "review"})
data["target"] = data.rating.map(lambda x: 1 if x >= 4 else 0)
data.head()

,review,rating,target
0,3D Touch просто восхитительная вещь! Заряд дер...,5,1
1,"Отключается при температуре близкой к нулю, не...",4,1
2,"В Apple окончательно решили не заморачиваться,...",3,0
3,Постарался наиболее ёмко и коротко описать все...,4,1
4,Достойный телефон. Пользоваться одно удовольст...,5,1


In [71]:
data.shape

(99, 3)

## Добавляем новую колонку - результат анализа тональности текста

In [72]:
data["pre_class"] = data.review.map(str).map(classify_text)
data.head()

,review,rating,target,pre_class
0,3D Touch просто восхитительная вещь! Заряд дер...,5,1,POSITIVE
1,"Отключается при температуре близкой к нулю, не...",4,1,NEGATIVE
2,"В Apple окончательно решили не заморачиваться,...",3,0,NEUTRAL
3,Постарался наиболее ёмко и коротко описать все...,4,1,NEUTRAL
4,Достойный телефон. Пользоваться одно удовольст...,5,1,POSITIVE


In [74]:
data.shape

(99, 4)

## Сохраняем результат отдельным файлом

In [77]:
data.to_csv('data/pre_classified_data.csv', index=False)